In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('fr.openfoodfacts.org.products.csv', sep='\t')

C:\Users\nberraba\AppData\Local\Temp\ipykernel_10024\2156673982.py:1: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fr.openfoodfacts.org.products.csv', sep='\t')


# Nettoyage des données

## Repérer variables pertinentes

In [3]:
df.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,4559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,16087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,16094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#dimensions du jeu de données

df.shape

(320772, 162)

In [11]:
#suppression colonnes vides

df.dropna(axis=1, how='all', inplace=True)

In [12]:
df.shape

(320772, 146)

L'objectif étant de connaître la qualité nutritionnelle des produits, nous ne garderont que:
 - le nom du produit
 - le nutriscore (nombre) français
 - le nutriscore (lettre) français
 - les valeurs nutritionnelles des produits

In [25]:
#liste de toutes les colonnes en rapport avec les valeurs nutritionnelles

val_nutri = [col for col in df.columns if col.endswith('_100g')]

In [26]:
# calcule du pourcentage de valeurs manquantes pour les valeurs nutritonnelles

pourcentage_valeurs_manquantes = (df[val_nutri].isnull().sum() / len(df)) * 100

print(f'Il y a en moyenne {round(pourcentage_valeurs_manquantes.mean(), 1)} % de valeurs manquantes par colonne.')

Il y a en moyenne 86.9 % de valeurs manquantes par colonne.


On voit qu'il manque en moyenne un grand nombre de données pour les colonnes sélectionnées.
Ajoutons donc d'autre colonnes qui pourraient nous aider à combler ses valeurs manquantes.

Dans un premier temps, faisons une large sélection avant de sélectionner plus précisémment.

In [143]:
categorie = ['categories', 'categories_fr', 'categories_tags', 'main_category', 'main_category_fr']
label = ['labels', 'labels_tags', 'labels_fr'] 
ingredient = ['ingredients_text','traces', 'traces_tags'] 
additif = ['additives_tags', 'additives','additives_fr', 'additives_n']
huile_palme =['ingredients_from_palm_oil_n','ingredients_from_palm_oil_tags',
              'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_tags']
nutriscore = ['nutrition_grade_fr']

In [144]:
# Fonction pour vérifier si au moins deux colonnes avec le même nom sont identiques
def colonnes_identiques(df, liste_colonnes):
    occurrences = {}
    for colonne in liste_colonnes:
        if colonne in occurrences:
            if any(df[colonne] == df[occurrences[colonne]]):
                return True
        else:
            occurrences[colonne] = colonne
    return False

In [149]:
#vérifions si des colonnes sont identiques dans chaque liste

colonnes_identiques(df, nutriscore)

False

In [150]:
colonnes_identiques(df, categorie)

False